In [ ]:
## (Clear)指定のキーワードが含まれるニュースのタイトルとURLを抽出し、保存する ##
# (Clear)「複数地域」「複数時間」の google news 検索結果ページで行う #
# (Clear)発布時間で並び替える #
#（Clear）指定のソース・日付の記事のみ表示 #
# (Clear)記事の概要にアクセスし、特定の記述が含まれるもののみ抽出 #
# 抽出された結果の横に「チェックボックス」をいれ、チェックされたもののみ再表示、保存する #

In [ ]:
import pandas as pd
import urllib
import requests
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
## 検索条件による検索結果ページのURLを確定し、辞書に格納する ##
google_news_url = "https://news.google.com"
dic_en = {
    "key_word":"self driving",
    "time":"1h", # Options:1h,1d,7d,1y,any
    "language":"ENG",
    "URL":"&hl=en-US&gl=US&ceid=US%3Aen"
}
dic_jp = {
    "key_word":"野球",
    "time":"1d",
    "language":"JPN",
    "URL":"&hl=ja&gl=JP&ceid=JP%3Aja"
}
dic_list = [dic_en,dic_jp]

url_dic = {}
for dic in dic_list:
    constraints = dic["key_word"] if dic["time"]=="any" else dic["key_word"] + " when:" + dic["time"]
    print(constraints)
    quote = urllib.parse.quote(constraints) # constraintsをURL中の形式に変換する
    URL = google_news_url+"/search?q="+quote+dic["URL"]
    print(URL)
    url_dic[dic["language"]] = URL
url_dic


In [ ]:
## 検査結果ページにある記事のタイトル・URL・概要・ソース・日付をスクレイプングする ##
def scrapping_news(language): # Options:"JPN","ENG"
    html = requests.get(url_dic[language]).text # htmlの取得
    bs = BeautifulSoup(html, 'html.parser') # htmlの解析
    span = bs.find_all("div",{"class":"xrnccd"}) # 記事の情報が格納される html 文を見つける #
    # 各記事の html 文からさらにタイトル・URL・ソースが格納される部分を抽出 #
    list_ = []
    columns=["Date","Title","Abstract","URL","Source"]
    df = pd.DataFrame(columns=columns)
    for tag in span:  # tagは個々記事のhtml文（タグ＝div,class=xrnccdの部分）を表し、それぞれに対して以下の処理を行う
        title_and_url = tag.find("a",{"class":"DY5T1d"}) # タイトル・URLが格納される文
        source = tag.find("a",{"class":"wEwyrc AVN2gc uQIVzc Sksgp"}) # ソースが格納される文
        time = tag.find("time",{"class":"WW6dff uQIVzc Sksgp"}) # 日付が格納される文
        abstract = tag.find("span",{"class":"xBbh9"}) # 概要が格納される文
        title = title_and_url.string # タイトルは文字部分を指定することで取得できる
        if title not in list_:
            list_.append(title)
            print(title)
            URL = google_news_url+title_and_url.get("href") # URLは属性：hrefに格納される
            print(URL)
            abstract = abstract.string # 概要は文字部分を指定することで取得できる
            print(abstract)
            source = source.string # ソースも文字部分を指定することで取得できる
            print("Source:"+source)
            datetime_ = time.get("datetime")[0:10].replace("-","/") # 日付は属性：datetimeに格納される
            print("Published Date:"+datetime_)
            se = pd.Series([datetime_,title,abstract,URL,source],columns)
            df = df.append(se,columns)
    return df
table_en = scrapping_news("ENG")
table_jp = scrapping_news("JPN")

In [ ]:
table_en
table_jp


In [ ]:
## スクレイプングした記事を日付の昇順・降順で並び替える ##
def sort_table_by_date(table,ascend=True):
    Date_str = table.loc[:,"Date"] # str形式の日付を保存する
    columns = table.columns # columnsを保存する（後でDataFrameを作る際に便利）

    list_ = []
    for i in table.values:
        list_.append(int(i[0].replace("/",""))) # sortするために、日付を数値形式に変え、リストに格納する
    table.loc[:,"Date"] = list_ # リストを使い、元tableの日付を数値化されたもので置き換える
    #（後で使うnp.argsortはnumpy配列のみに有効だから、新tableは作らなければならない）#

    # 新tableの日付の部分を抽出し並び替え、その結果のインデックスを取得する
    if ascend:
        index_sorted = np.argsort(table.values[:,0],axis=0)
    else:
        index_sorted = np.argsort(table.values[:,0],axis=0)[::-1]

    table.loc[:,"Date"] = Date_str # str形式の日付を使い元tableに還元する
    # 元tableのvalues部分を新インデックスに従い並び替え、それに基づき最終のsortしたtableを作る
    table_sorted = pd.DataFrame(table.values[index_sorted],columns=columns)
    return table_sorted

In [ ]:
table_jp_sorted = sort_table_by_date(table_jp,False)
table_jp_sorted


In [ ]:
## 指定のソース・日付の記事のみ表示させる ##
dic_condition_jp = {
    "condition1":None,
    "condition2":"Yahoo!ニュース"
}
def pick_up_by_source_or_time(table,dic):
    df = pd.DataFrame(columns=table.columns)
    columns = list(table.columns)
    # 適用される条件を確認する（Date,Source共に制限するか、一方のみか）#
    if (dic["condition1"]!=None) & (dic["condition2"]==None): # Dateのみ
        for i in table.values:
            if i[0]==dic["condition1"]: # Dateで判断
                    print(i[1])
                    print(i[2])
                    print(i[3])
                    print("Source:"+i[4])
                    print("Published Date:"+i[0])
                    se = pd.Series([i[0],i[1],i[2],i[3],i[4]],columns)
                    df = df.append(se,columns)
            else:
                continue
    elif (dic["condition1"]==None) & (dic["condition2"]!=None): # Sourceのみ
        for i in table.values:
            if i[4]==dic["condition2"]: # Sourceで判断
                    print(i[1])
                    print(i[2])
                    print(i[3])
                    print("Source:"+i[4])
                    print("Published Date:"+i[0])
                    se = pd.Series([i[0],i[1],i[2],i[3],i[4]],columns)
                    df = df.append(se,columns)
            else:
                continue
    elif (dic["condition1"]!=None) & (dic["condition2"]!=None): # 両方
        for i in table.values:
            if i[4]==dic["condition2"] & i[0]==dic["condition1"]: # 両方同時判断
                    print(i[1])
                    print(i[2])
                    print(i[3])
                    print("Source:"+i[4])
                    print("Published Date:"+i[0])
                    se = pd.Series([i[0],i[1],i[2],i[3],i[4]],columns)
                    df = df.append(se,columns)
            else:
                continue
    return df
table_jp_picked_by_source_or_time = pick_up_by_source_or_time(table=table_jp,dic=dic_condition_jp)
table_jp_picked_source_or_time


In [ ]:
## 記事の概要にアクセスし、特定の記述が含まれるもののみ抽出 ##
key_words_list_jp = ["緊急事態宣言","プロ","コロナ","ホームラン"]

def pick_up_by_contents(table,key_words_list):
    list_ = []
    df = pd.DataFrame(columns=table.columns)
    columns = list(table.columns)
    for word in key_words_list:
        for i in table.values:
            if (word in i[2]) & (i[1] not in list_):
                list_.append(i[1])
                print(i[1])
                print(i[2])
                print(i[3])
                print("Source:"+i[4])
                print("Published Date:"+i[0])
                se = pd.Series([i[0],i[1],i[2],i[3],i[4]],columns)
                df = df.append(se,columns)
            else:
                continue
    return df
table_jp_picked_by_contents = pick_up_by_contents(table=table_jp,key_words_list=key_words_list_jp)
table_jp_picked_by_contents
